https://blog.csdn.net/u012162613/article/details/42177327

# PCA 算法步骤
- 零均值化
- 求协方差矩阵
- 求特征值、特征矩阵
- 保留主要成分[即保留值比较大的前n个特征]

In [119]:
import numpy as np

def zero_mean(data_mat):
    """
    零均值化
    """
    mean_val = np.mean(data_mat, axis = 0) #按列求均值，即求各个特征的均值
    new_data = data_mat - mean_val
    return new_data, mean_val

def pca(data_mat, n):
    new_data, mean_val = zero_mean(data_mat)
    cov_mat = np.cov(new_data, rowvar = 0)  #求协方差矩阵,return ndarray；若rowvar非0，一列代表一个样本，为0，一行代表一个样本
    
    eig_vlas, eig_vects = np.linalg.eig(np.mat(cov_mat)) #求特征值和特征向量,特征向量是按列放的，即一列代表一个特征向量
    eig_val_indice = np.argsort(eig_vlas) #对特征值从小到大排序
    n_eig_val_indice=eig_val_indice[-1:-(n+1):-1]   #最大的n个特征值的下标( [::-1]是逆序的意思)
    n_eig_vects = eig_vects[:,n_eig_val_indice] #最大的n个特征值对应的特征向量
    low_data_mat=new_data*n_eig_vects               #低维特征空间的数据(点乘)
    recon_mat = (low_data_mat * n_eig_vects.T) + mean_val #重构数据
    return low_data_mat, recon_mat   
     
     

a = np.array([
    [1, 4, 5, 7, 9],
    [1, 5, 4, 7, 5],
    [3, 4, 6, 8, 9]
])

l1, l2 = pca(a, 1)
print(l1)
print(l2) 
 

print('OK')


[[-0.96388791]
 [ 3.0069765 ]
 [-2.0430886 ]]
[1.66666667 4.33333333 5.         7.33333333 7.66666667]
[[-0.28887379]
 [ 0.21257931]
 [-0.3570162 ]
 [-0.1444369 ]
 [-0.85031722]]
OK


## 选择主成分个数
    文章写到这里还没有完，应用PCA的时候，对于一个1000维的数据，我们怎么知道要降到几维的数据才是合理的？即n要取多少，才能保留最多信息同时去除最多的噪声？一般，我们是通过方差百分比来确定n的，这一点在Ufldl教程中说得很清楚，并且有一条简单的公式，下面是该公式的截图：
   ![301](picture/301.jpeg)

In [108]:
import numpy as np

def zero_mean(data_mat):
    """
    零均值化
    """
    mean_val = np.mean(data_mat, axis = 0) #按列求均值，即求各个特征的均值
    new_data = data_mat - mean_val
    return new_data, mean_val

def percentage2n(eig_vals, percentage):
    sort_array = np.sort(eig_vals)   #升序
    sort_array = sort_array[-1::-1]  #逆转，即降序
    array_sum = sum(sort_array)
    
    tmp_sum = 0
    num = 0
    for item in sort_array:
        tmp_sum += item
        num += 1
        if tmp_sum >= array_sum*percentage:
            return num
    return num

def pcaa(data_mat, percen_tage=0.99):
    new_data, mean_val = zero_mean(data_mat)
    cov_mat = np.cov(new_data, rowvar = 0)  #求协方差矩阵,return ndarray；若rowvar非0，一列代表一个样本，为0，一行代表一个样本
    
    eig_vlas, eig_vects = np.linalg.eig(np.mat(cov_mat)) #求特征值和特征向量,特征向量是按列放的，即一列代表一个特征向量
    eig_val_indice = np.argsort(eig_vlas) #对特征值从小到大排序
    
    n = percentage2n(eig_vlas, percen_tage)
    
    n_eig_val_indice=eig_val_indice[-1:-(n+1):-1]   #最大的n个特征值的下标( [::-1]是逆序的意思)
    n_eig_vects = eig_vects[:,n_eig_val_indice] #最大的n个特征值对应的特征向量
    low_data_mat=new_data*n_eig_vects               #低维特征空间的数据(点乘)
    recon_mat = (low_data_mat * n_eig_vects.T) + mean_val #重构数据
    return low_data_mat, recon_mat  

a = np.array([
    [1, 4, 5, 7, 9],
    [1, 5, 4, 7, 5],
    [3, 4, 6, 8, 9]
])

l1, l2 = pcaa(a, 0.5)
print(l1)
print(l2) 

print('OK')

[[-0.96388791]
 [ 3.0069765 ]
 [-2.0430886 ]]
[[1.94510862 4.12843071 5.3441236  7.47255431 8.48627716]
 [0.79802995 4.97255431 3.92646067 6.89901498 5.10978275]
 [2.25686142 3.89901498 5.72941573 7.62843071 9.40394009]]
OK


In [60]:
from sklearn.decomposition import PCA


In [118]:

#定义PCA算法               
def PCA(data,r):
    data=np.float32(np.mat(data))
    rows,cols=np.shape(data)
    data_mean=np.mean(data,0)#对列求平均值
    A=data-np.tile(data_mean,(rows,1))#将所有样例减去对应均值得到A
    C=A*A.T #得到协方差矩阵
    D,V=np.linalg.eig(C)#求协方差矩阵的特征值和特征向量 
    V_r=V[:,0:r]      # 按列取前r个特征向量
    V_r=A.T*V_r#小矩阵特征向量向大矩阵特征向量过渡
    for i in range(r):
        V_r[:,i]=V_r[:,i]/np.linalg.norm(V_r[:,i])#特征向量归一化
    
    final_data=A*V_r
    return final_data,data_mean,V_r


a = np.array([
    [1, 4, 5, 7, 9],
    [1, 5, 4, 7, 5],
    [3, 4, 6, 8, 9]
])

l1, l2, l3 = PCA(a, 1)
print(l1)
print(l2)
print(l3)

[[-0.96388805]
 [ 3.0069766 ]
 [-2.043089  ]]
[[1.6666666 4.3333335 5.        7.3333335 7.6666665]]
[[-0.2888738 ]
 [ 0.21257932]
 [-0.35701624]
 [-0.1444369 ]
 [-0.85031724]]
